## Clean film script dataset and find matching IMDb_id and tmdb_id

Description: 
- load in raw dataset of film scripts obtained from webscrapers
- clean any inconsistencies or mistakes
- assign each film script an IMDb_id and tmdb_id

This script is meant to be run only one time (i.e. when the data was first webscraped). Further updating should be done using the data_updating notebook which is integrated into the data pipeline

In [ ]:
import pandas as pd
import numpy as np
import requests
from matplotlib import pyplot as plt
import pickle
import time

In [ ]:
# import raw (uncleaned) film script dataset
path_to_csv = (
    "scraped_scripts\\springfield_movie_scripts_2023_01_13.csv"
)
df = pd.read_csv(path_to_csv, index_col=[0])

df.head()

In [ ]:
# create Movie Year feature using information from Movie Title column
df["Movie Year"] = df["Movie Title"].map(lambda x: x[-5:-1])
df["Movie Title"] = df["Movie Title"].map(lambda x: x[0:-6])

# manually inspect and fix data errors such as movie being released in year 0000 or 0147
print(df["Movie Year"].unique())
print()
print(df.loc[df["Movie Year"] == "0147", ["Movie Title", "Movie Year"]])

# manually go in and fix the spotted errors. Use IMDb to find correct information
df.loc[12856, "Movie Title"] = "Hollows Grove"
df.loc[12224, "Movie Title"] = "Alien Battlefield Earth"
df.loc[8151, "Movie Title"] = "Disappearance"

df.loc[df["Movie Year"] == "0000", "Movie Year"] = "1986"
df.loc[df["Movie Year"] == "2050", "Movie Year"] = "2017"
df.loc[df["Movie Year"] == "0147", "Movie Year"] = "2014"
df.loc[df["Movie Year"] == "arth", "Movie Year"] = "2021"

# df_orig["Movie Year"] = df_orig["Movie Year"].astype("int")

# reorder and rename columns
df = df[["Movie Title", "Movie Year", "Movie Script Link", "Script Text"]]
df.columns = ["movie_title", "movie_year", "springfield_link", "script_text"]
df.head()

In [ ]:
# clean up title names: remove white space and convert 'title, the' to 'the title'
df["movie_title"] = df["movie_title"].str.strip()
df["movie_title"] = [
    "The " + jmovie_title[0:-5] if jmovie_title.endswith(", The") else jmovie_title
    for jmovie_title in df["movie_title"]
]

In [ ]:
# inspect movies with duplicate name and year
duplicates = df[["movie_title", "movie_year"]].value_counts()
duplicates = duplicates[duplicates > 1]
print(f'Number of duplicates: {len(duplicates)}')
for jmovie in duplicates.index:
    print(df.query("movie_title == @jmovie[0] and movie_year == @jmovie[1]"))
    

In [ ]:
# The door and the avengers are not duplicates but have the same name and year
# drop all other duplicates from the database and reset the index
jdrop = []
for j in duplicates.index:
    if j[0].lower() == "the door" or j[0].lower() == "the avengers":
        print(j[0])
    else:
        jdrop.append(df.query("movie_title == @j[0] and movie_year == @j[1]").index[1])
df = df.drop(jdrop, axis=0)
df = df.reset_index(drop=True)

In [ ]:
# drop movies with less than 75 words
# these scripts contain either none or "More Movie Scripts | Request a Movie Transcript"
df["num_words"] = [len(jscript) for jscript in df["script_text"]]
df = df.drop(df[df["num_words"] < 75].index, axis=0)
df = df.drop("num_words", axis=1)
df = df.reset_index(drop=True)

## Synthesize film script dataset with IMDb dataset by assigning each movie in film script dataset an IMDb_id

### method a) match tables based on title name

In [ ]:
# make a copy of original dataframe 
df_orig = df.copy()

# Now preprocess new movie script dataframe titles
df['movie_title'] = df['movie_title'].str.lower()
df['movie_title'] = df['movie_title'].str.strip()
df['movie_year'] = df['movie_year'].astype(int)

In [ ]:
# load in IMDB database
path_to_tsv = 'C:\\Users\\Nick\\Documents\\DataScience\\databases\\IMDB_database\\2023_02_12_IMDbDataFiles\\titleBasics.tsv'
df_imdb = pd.read_csv(path_to_tsv, sep='\t')

# preprocess imdb data titles
df_imdb['primaryTitle'] = df_imdb['primaryTitle'].str.lower()
df_imdb['originalTitle'] = df_imdb['originalTitle'].str.lower()
df_imdb['startYear'] = df_imdb['startYear'].replace('\\N','0').astype(int)

In [ ]:
# iterate through all rows (movie scripts) of dataframe
# and attempt to find a matching title and/or year in the script dataframe
# with one in the imdb dataset. If no match is found or if several matches are found
# return 'NULL'. We will return to them later. The following code correctly matches
# about 95% of movies and took 272 minutes to run this code cell
tt_all = []
year_all = []
jredo = []
for j in range(len(df)):
    
    ij = df_imdb['primaryTitle'] == df.loc[j,'Movie Title']
    if(np.any(ij)):
        if(np.sum(ij) == 1):
            tt_all.append(df_imdb.loc[ij,'tconst'].values[0])
            year_all.append(df_imdb.loc[ij,'startYear'].values[0])
        if(np.sum(ij) > 1):
            # filter by titleType and year
            dfyear = df.loc[j,'Movie Year']
            dfyearp = dfyear+1
            dfyearm = dfyear-1
            df_title_year = df_imdb[ij].query("(titleType == 'movie' or titleType == 'tvMovie' or titleType == 'video') and (startYear == @dfyear or startYear == @dfyearp or startYear == @dfyearm)")
            if(len(df_title_year) == 1):
                tt_all.append(df_title_year['tconst'].values[0])
                year_all.append(df_title_year['startYear'].values[0])
            elif(len(df_title_year) > 1):
                if(len(df_title_year.query("titleType == 'movie' or titleType == 'tvMovie'")) == 1):
                    temp = df_title_year.query("titleType == 'movie' or titleType == 'tvMovie'")
                    tt_all.append(temp['tconst'].values[0])
                    year_all.append(temp['startYear'].values[0])                   
                else:
                    tt_all.append([j for j in df_title_year['tconst'].values[:]])
                    year_all.append(df_title_year['startYear'].values[0])
            else:
                jredo.append(j)
                tt_all.append('NULL')
                year_all.append('NULL')
    elif(np.any(df_imdb['originalTitle'] == df.loc[j,'Movie Title'])):
        ij = df_imdb['originalTitle'] == df.loc[j,'Movie Title']
        if(np.sum(ij) == 1):
            tt_all.append(df_imdb.loc[ij,'tconst'].values[0])
            year_all.append(df_imdb.loc[ij,'startYear'].values[0])
        if(np.sum(ij) > 1):
            # filter by titleType and year
            dfyear = df.loc[j,'Movie Year']
            dfyearp = dfyear+1
            dfyearm = dfyear-1
            df_title_year = df_imdb[ij].query("(titleType == 'movie' or titleType == 'tvMovie' or titleType == 'video') and (startYear == @dfyear or startYear == @dfyearp or startYear == @dfyearm)")
            if(len(df_title_year) == 1):
                tt_all.append(df_title_year['tconst'].values[0])
                year_all.append(df_title_year['startYear'].values[0])
            elif(len(df_title_year) > 1):
                if(len(df_title_year.query("titleType == 'movie' or titleType == 'tvMovie'")) == 1):
                    temp = df_title_year.query("titleType == 'movie' or titleType == 'tvMovie'")
                    tt_all.append(temp['tconst'].values[0])
                    year_all.append(temp['startYear'].values[0])                   
                else:
                    tt_all.append([j for j in df_title_year['tconst'].values[:]])
                    year_all.append(df_title_year['startYear'].values[0])
            else:
                jredo.append(j)
                tt_all.append('NULL')
                year_all.append('NULL')
    else:
        jredo.append(j)
        tt_all.append('NULL')
        year_all.append('NULL')

In [ ]:
# add tconst to original dataframe 
df_orig['imdb_id'] = tt_all

# for multiple tconsts, keep only the first one in the list
df_orig['imdb_id'] = [jk.replace('[','').replace(']','').replace("'",'').split(', ')[0] if "['tt" in jk else jk for jk in df_orig['imdb_id'].tolist()]

# create IMDb link to movie based on imdb_id
df_orig['imdb_link'] = 'https://www.imdb.com/title/' + df_orig['imdb_id'] + '/'

# CHECKPOINT SAVE - save updated dataset that has been cleaned and contains IMDb_id
# df_orig.to_csv('data_cleaning_and_synthesis_out\\springfield_movie_scripts_2023_01_13_clean.csv')

## Synthesize film script dataset with tmdb dataset by assigning each movie in film scripts dataset the corresponding tmdb_id. Use tmdb API

### methoda a) call tmdb api using imdb_id

In [ ]:
tmdb_poster = []
imdb_id2 = []
tmdb_id = []
count = 0
for j, jtt in enumerate(df_orig["imdb_id"]):
    url = "https://api.themoviedb.org/3/movie/{}?api_key=075d83b3063def6fdd12763959a9086e&language=en-US".format(
        jtt
    )

    if j % 2000 == 0:
        print(j)
    try:
        data = requests.get(url)
        data.raise_for_status()
        data = data.json()
        tmdb_poster.append('https://image.tmdb.org/t/p/w500/' + data["poster_path"])
        tmdb_id.append(data["id"])
        imdb_id2.append(data["imdb_id"])
    except:
        tmdb_poster.append("NULL")
        tmdb_id.append(np.nan)
        imdb_id2.append(np.nan)
        count = count + 1
        # if(count%25 == 0):
        #     print(count)

In [ ]:
# update df_orig
df_orig['tmdb_id'] = tmdb_id
df_orig['tmdb_poster_link'] = tmdb_poster
df_orig['imdb_id2'] = imdb_id2

In [ ]:
# test to make sure that tmdb reported imdb_id matches imdb_id from IMDb dataset
df_test = df_orig.query("tmdb_poster_link != 'NULL'").copy()
print(len(df_test))
print(np.sum(df_test['imdb_id'] != df_test['imdb_id2']))
df_orig = df_orig.drop('imdb_id2', axis = 1)

### method b) call tmdb (search) api using movie name

In [ ]:
tmdb_id = []
tmdb_poster = []
df_copy = df_orig[df_orig['tmdb_poster_link'] == 'NULL'].copy()
for j, movie_title in enumerate(df_copy['movie_title']): 
    url = "http://api.themoviedb.org/3/search/movie?api_key=075d83b3063def6fdd12763959a9086e&query={}".format(
        movie_title.replace(" ", "+")
    )
    if(j%500 == 0):
        print(j)
    try:
        data = requests.get(url)
        data.raise_for_status()
        data = data.json()
        if(data['total_results'] < 1):
            tmdb_id.append("NULL")
            tmdb_poster.append("NULL")           
        elif(data['total_results'] < 2):
            tmdb_id.append(data['results'][0]['id'])
            tmdb_poster.append(data['results'][0]['poster_path'])
        else:
            tmdb_id.append("NULL")
            tmdb_poster.append("NULL")    
            # print('input movie:' + movie_title)
            # for jj, jmovie in enumerate(data['results']):
            #     print(str(jj) + ': ' + jmovie['title'])

            # time.sleep(1)
            # user_input = int(input("enter a value (-1 returns null): "))
            # if user_input >= 0:
            #     tmdb_id.append(data['results'][user_input]['id'])
            #     tmdb_poster.append(data['results'][user_input]['poster_path'])
            # else:
            #     tmdb_id.append("NULL")
            #     tmdb_poster.append("NULL")    
    except:
        tmdb_id.append("NULL")
        tmdb_poster.append("NULL")       

In [ ]:
# update df_orig
tmdb_poster = [j if type(j) == str else 'NULL' for j in tmdb_poster]
df_orig.loc[df_copy.index,'tmdb_poster_link'] = ['https://image.tmdb.org/t/p/w500/' + jposter if jposter != 'NULL' else 'NULL' for jposter in tmdb_poster]
df_orig.loc[df_copy.index,'tmdb_id'] = tmdb_id

In [ ]:
# finally, attempt to back out imdb id if imdb_id is null but tmdb_id is known
# this method works well for movies that have quirky spelling or had title changes

df_orig['imdb_id'] = df_orig['imdb_id'].fillna('NULL')
df_copy = df_orig.query("imdb_id == 'NULL' and tmdb_id != 'NULL'").copy()

tmdb_poster = []
imdb_id2 = []
tmdb_id = []
for j, jtt in enumerate(df_copy["tmdb_id"]):
    url = "https://api.themoviedb.org/3/movie/{}?api_key=075d83b3063def6fdd12763959a9086e&language=en-US".format(
        jtt
    )

    if j % 2000 == 0:
        print(j)
    try:
        data = requests.get(url)
        data.raise_for_status()
        data = data.json()
        tmdb_poster.append('https://image.tmdb.org/t/p/w500/' + data["poster_path"])
        tmdb_id.append(data["id"])
        imdb_id2.append(data["imdb_id"])
    except:
        tmdb_poster.append("NULL")
        tmdb_id.append(np.nan)
        imdb_id2.append(np.nan)

In [ ]:
# update df_orig
imdb_id2 = [j if type(j) == str else 'NULL' for j in imdb_id2]
imdb_id2 = ['NULL' if j == '' else j for j in imdb_id2]
df_orig.loc[df_copy.index,'imdb_id'] = imdb_id2
df_orig.loc[df_copy.index,'imdb_link'] = 'https://www.imdb.com/title/' + df_copy['imdb_id2'] + '/'

In [ ]:
# do some final type casting for consistent outputs
df_orig['tmdb_id'] = df_orig['tmdb_id'].fillna(-1).astype(int)
df_orig.loc[df_orig['imdb_id'] == 'nan','imdb_id'] = 'NULL'
df_orig['imdb_link'] = df_orig['imdb_link'].fillna('NULL')
df_orig['tmdb_poster_link'] = df_orig['tmdb_poster_link'].fillna('NULL')
df_orig['imdb_link'] = ['NULL' if j == 'https://www.imdb.com/title/NULL/' else j for j in df_orig['imdb_link']]
df_orig.info()

In [ ]:
# FINAL SAVE -- ALL OTHER CHANGES TO DATAFRAME WILL BE DONE INPLACE USING data_updating.ipynb notebook
df_orig.to_csv(
    "data_cleaning_and_synthesis_out\\springfield_movie_scripts_2023_01_13_clean.csv"
)